In [1]:
import pandas as pd
import networkx as nx
from pyvis import network as net
from pyvis.network import Network
import random

from serpapi import GoogleScholarSearch
import json
import requests

## ES 1
A partire da nodes.csv, utilizzare la libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati:
- author_id: ID identificativo del profilo Google Scholar
- cited_by: numero totale di citazioni ricevute
- interests: elenco degli interessi di ciascun autore
Sfruttando la libreria Python Pandas, usare la struttura dati DataFrame per aggiornare il file originale con apposite colonne e memorizzarlo nella cartella /data.

In [12]:
apikey ={
    'SERP_API_KEY_FAVA': "1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048", 
    'SERP_API_KEY_LIGUGNANA': "11a077a8bee5bb1c2a25af914046bc7ce53050b3416e5720239e43223c63374b",
    'SERP_API_KEY_PARAVANO' :  "",
    'SERP_API_KEY_TAMMARO': "6eaf5a9a0d2eef8bf026a947a3dffbb0d31cb1d6e122b81c0e6062f6b23a4c40"
}

def key_choice():
    return random.choice(list(apikey.items()))[1]
#for che cambi ala key se da errore la chiamata


In [13]:
key_choice()

'11a077a8bee5bb1c2a25af914046bc7ce53050b3416e5720239e43223c63374b'

In [15]:
def list_profile(apiKey, name, affiliation):
    base_url = "https://serpapi.com/search"
    
    params = {
        "api_key": apiKey,
        "engine": "google_scholar_profiles",
        "hl": "en",
        "mauthors": name
    }      
    
    response = requests.get(base_url, params = params)
    results = response.json()
    
    authorId = results["profiles"][0]["author_id"] #save author_id from of the author
    citedBy = results["profiles"][0]["cited_by"]
    interests = results["profiles"][0]["interests"]
    
    #verifies the affiliation, the profile found is the one that i'm searching for
    if results["profiles"][0]["affiliations"] == affiliation:
       tupla=tuple((name,affiliation,authorId,citedBy,interests))
       df = pd.DataFrame({'name': [tupla[0]],
                   'affiliations': [tupla[1]],
                   'author_id': [tupla[2]],
                   'cited_by': [tupla[3]],
                   'interests': [tupla[4]]})
       return df
   
        

In [ ]:
df_nodes = pd.read_csv(r"Data/nodes.csv")
for index, row in df_nodes.iterrows():
    ##list_profile(apikey, row[0], row[1])
    
df_nodes2 = df_nodes
df_nodes2 = pd.DataFrame(df_nodes2, columns=)


authorId = "author_id" #save author_id from of the author
citedBy = "cited_by"
interests = "interests"


In [ ]:
def author_profile(apiKey, authorID, affiliation):
    
    base_url = "https://serpapi.com/search"
    
    params = {
        "api_key": apiKey,
        "engine": "google_scholar_author",
        "hl": "en",
        "author_id": authorID
    }
    
    response = requests.get(base_url, params = params)
    results = response.json()
    
    return #dataset

In [16]:
'''
df_nodes = pd.read_csv(r"Data/nodes.csv")
file = open('all_profiles_for_larry_page.json')
res = json.load(file)
'''

list_profile("1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048", 'David La Barbera', 'Università Degli Studi Di Udine')

,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,"[{'title': 'Computer Science', 'serpapi_link':..."
